In [66]:
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.models import Sequential

In [67]:
import numpy as np
import gensim,jieba,os

In [68]:
from gensim.models import Word2Vec
w =  Word2Vec.load('./self_train_word2vec')


## 词向量

In [69]:
def load_embedding(w):
    index = 0
    word2id = {}
    vec = []
    for word in w.wv.vocab.keys():
        word2id[word] = index
        vec.append(w[word])
        index +=1
    #add two words
    word2id['UNK'] = len(word2id) # 位置字符代替id
    word2id['BLANK'] = len(word2id) # 临时填充
    vec.append(np.random.normal(size=100, loc=0, scale=0.05))
    vec.append(np.random.normal(size=100, loc=0, scale=0.05))
    vec = np.array(vec, dtype=np.float32) #[[100]...]
    vec_dict = {}
    for i in range(vec.shape[0]):
        vec_dict[i] = vec[i]
    return vec,word2id

In [70]:
vec,word2id = load_embedding(w)
print(vec.shape,len(word2id))

(2524, 100) 2524


/Users/lucas/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel/__main__.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# X  
## 转换成句子id   [id,id,id,id....]-生成x

In [71]:
# df = pd.read_csv('cleandemo8.csv')
# df = pd.read_csv('data_labeled_2.csv')
df =pd.read_csv('./df_cluster.csv')
# df = pd.read_csv('./clustering_withKeyword.csv')
# df = pd.read_csv('/Users/lucas/Desktop/workspace/IntentionClassify/clustering_withLSTM.csv')
# df = pd.read_csv('/Users/lucas/Desktop/workspace/IntentionClassify/clustering_withIDF.csv')
sens_vec = []
black = np.array([0]*w['话费'].shape[0])
data_word = []
fix_lenght = 20
c = 1
for sen in df.request:
    sen_vec = [word2id[word] if word in word2id else word2id['UNK'] for word in jieba.cut(sen)]
    sen_vec = sen_vec[:fix_lenght]
    while len(sen_vec) < fix_lenght:
        sen_vec.append(word2id['BLANK'])   
    data_word.append(sen_vec)
    c+=1
    if c>10000:
        break
data_word = np.array(data_word)

/Users/lucas/anaconda/envs/py3/lib/python3.6/site-packages/ipykernel/__main__.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


# y

In [72]:
# df = pd.read_csv('./data_labeled_2.csv')
print(df.shape)
df.head(2)

(6132, 4)


,sessionid,request,response,cluster
0,13410000258T16081911402229APP,实名制了怎么还不能打电话？,请问是本机吗？我帮您开机，请稍等哈,2
1,13410000422T16082318212113APP,查询我现在的套餐,，您改了38元4G飞享套餐，9-13号生效哦，还在线吗，还有其他可以帮到您吗？,3


## label

In [73]:
name = set(df.cluster)
label_dict = dict(zip(name,range(len(name))))
label_ids = [label_dict[word] for word in df.cluster]
y = np.eye(len(label_dict))[label_ids]
print(y[0],label_ids[0],len(label_dict))

[0. 0. 1. 0. 0. 0. 0.] 2 7


In [74]:
from keras.layers.wrappers import TimeDistributed
from keras.layers import Flatten,Activation
import matplotlib.pyplot as plt
from keras.layers import Bidirectional

# split train and test

In [75]:
cut = int(data_word.shape[0]*0.7)
# tr_x = data_word[:cut]
# tr_y = y[:cut]
# ts_x = data_word[cut:]
# ts_y = y[cut:]

from sklearn.model_selection import train_test_split
tr_x, ts_x, tr_y, ts_y = train_test_split(data_word, y, test_size=0.33, random_state=42)


```python
model = Sequential()
vocab_size = len(vec) 
model.add(Embedding(vocab_size, 100, weights=[vec], input_length=20, trainable=False))
model.add(Bidirectional(LSTM(25,dropout=0.4,return_sequences=True)))
model.add(TimeDistributed(Dense(24, activation='relu')))
model.add(Flatten())
model.add(Dense(14, activation='sigmoid'))
model.add(Dense(7, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

```

In [96]:
model = Sequential()
"""
All that the Embedding layer does is to map the integer inputs to the vectors found at the corresponding index 
in the embedding matrix, i.e. the sequence [1, 2] would be converted to [embeddings[1], embeddings[2]]. 
This means that the output of the Embedding layer will be a 3D tensor of shape 
(samples, sequence_length, embedding_dim)
"""
vocab_size = len(vec) # 最大的词数，与词向量实际数目有关
model.add(Embedding(vocab_size, 100, weights=[vec], input_length=20, trainable=False))
# model.add(Bidirectional(LSTM(25,dropout=0.4,return_sequences=True),merge_mode='concat'))
model.add(LSTM(25,dropout=0.4,return_sequences=True))
# model.add(Activation('softmax')) #this guy here
model.add(TimeDistributed(Dense(24, activation='relu')))
model.add(Flatten())
model.add(Dense(14, activation='sigmoid'))
model.add(Dense(7, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print(model.summary())


# model.add((LSTM(50,dropout=0.3,kernel_initializer='random_normal',input_shape=(100,10))))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
# fit the model
# data_words = data_word[:100]
# ys = np.reshape(y[:100],(y[:100].shape[0],1,1))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 100)           252400    
_________________________________________________________________
lstm_8 (LSTM)                (None, 20, 25)            12600     
_________________________________________________________________
time_distributed_8 (TimeDist (None, 20, 24)            624       
_________________________________________________________________
flatten_7 (Flatten)          (None, 480)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 14)                6734      
_________________________________________________________________
dense_24 (Dense)             (None, 7)                 105       
Total params: 272,463
Trainable params: 20,063
Non-trainable params: 252,400
_________________________________________________________________

In [97]:
ts_x.shape

(2024, 20)

In [98]:
history = model.fit(tr_x, tr_y, epochs=20, verbose=1,batch_size=50,validation_data=(ts_x,ts_y))
# evaluate the model

# print(history.history['categorical_accuracy'])
# loss, accuracy = model.evaluate(ts_x, ts_y, verbose=0)
# print('Accuracy: %f' % (accuracy*100))


Train on 4108 samples, validate on 2024 samples
Epoch 1/20
4108/4108 [==============================] - 4s 992us/step - loss: 1.8193 - categorical_accuracy: 0.2821 - val_loss: 1.7556 - val_categorical_accuracy: 0.3192
Epoch 2/20
4108/4108 [==============================] - 2s 592us/step - loss: 1.7462 - categorical_accuracy: 0.3186 - val_loss: 1.7105 - val_categorical_accuracy: 0.3552
Epoch 3/20
4108/4108 [==============================] - 2s 533us/step - loss: 1.7145 - categorical_accuracy: 0.3354 - val_loss: 1.6810 - val_categorical_accuracy: 0.3646
Epoch 4/20
4108/4108 [==============================] - 2s 539us/step - loss: 1.6937 - categorical_accuracy: 0.3444 - val_loss: 1.6695 - val_categorical_accuracy: 0.3622
Epoch 5/20
4108/4108 [==============================] - 2s 504us/step - loss: 1.6740 - categorical_accuracy: 0.3525 - val_loss: 1.6599 - val_categorical_accuracy: 0.3617
Epoch 6/20
4108/4108 [==============================] - 2s 480us/step - loss: 1.6678 - categorical_acc

# 评价

In [99]:
res = model.predict_classes(ts_x)
ts_y_classes = [list(x).index(1) for x in ts_y]
np.unique(ts_y_classes,return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6]), array([192, 260, 325, 306, 191, 646, 104]))

In [100]:
from sklearn.metrics import f1_score,accuracy_score,classification_report
print(accuracy_score(ts_y_classes,res))
print(classification_report(ts_y_classes,res))

0.3784584980237154
             precision    recall  f1-score   support

          0       0.40      0.09      0.15       192
          1       0.37      0.27      0.31       260
          2       0.22      0.11      0.15       325
          3       0.33      0.13      0.18       306
          4       0.39      0.45      0.42       191
          5       0.40      0.79      0.53       646
          6       0.50      0.05      0.09       104

avg / total       0.36      0.38      0.32      2024



Bseline：  acy: 0.4408 - val_loss: 1.6413 - val_categorical_accuracy: 0.3715
加权：acy: 0.6254 - val_loss: 1.1968 - val_categorical_accuracy: 0.5860 
复杂模型：acy: 0.6741 - val_loss: 1.2125 - val_categorical_accuracy: 0.6472
tf-idf：acy: 0.7047 - val_loss: 0.7166 - val_categorical_accuracy: 0.7085



In [81]:
history.history.keys()

dict_keys(['val_loss', 'val_categorical_accuracy', 'loss', 'categorical_accuracy'])

In [82]:
np.unique(df.cluster, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([ 561,  757, 1039,  920,  608, 1936,  311]))

In [83]:
np.unique(res, return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6]),
 array([  78,  183,  138,  186,  275, 1157,    7]))

In [84]:
count = 0
for i in range(len(ts_y_classes)):
    if ts_y_classes[i]==res[i]:
        count+=1
print(count,count/len(ts_y_classes))

752 0.3715415019762846


```python
model = Sequential()

vocab_size = len(vec) # 最大的词数，与词向量实际数目有关
model.add(Embedding(vocab_size, 100, weights=[vec], input_length=20, trainable=False))
# model.add(LSTM(5,kernel_initializer='random_normal',input_shape=(20,100),return_sequences=True))
model.add(LSTM(5,dropout=0.3,return_sequences=True))
# model.add(Flatten())
# model.add(TimeDistributed(Dense(5)))
# model.add(TimeDistributed(Dense(2, activation='sigmoid')))
model.add(Dense(15, activation='softmax'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())


# model.add((LSTM(50,dropout=0.3,kernel_initializer='random_normal',input_shape=(100,10))))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# print(model.summary())
```

In [85]:
# # fit the model
# data_words = data_word[:100]
# ys = np.reshape(y[:100],(y[:100].shape[0],1,1))
# model.fit(data_words, ys, epochs=20, verbose=0,batch_size=10)
# # evaluate the model
# loss, accuracy = model.evaluate(data_words, ys, verbose=0)
# print('Accuracy: %f' % (accuracy*100))

In [86]:
y[:100].shape

(100, 7)

In [87]:
# model.fit(sens_vec,y)

# 参考代码

In [88]:
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = np.array([1,1,1,1,1,0,0,0,0,0])
# integer encode the documents
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

NameError: name 'one_hot' is not defined

In [ ]:
from keras.utils import to_categorical
# define example
data = [1, 3, 2, 0, 3, 2, 2, 1, 0, 1]
data = np.array(data)
print(data)
# one hot encode
encoded = to_categorical(data)
print(encoded)

```python
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
# define documents
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']
# define class labels
labels = array([1,1,1,1,1,0,0,0,0,0])
# prepare tokenizer
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
# integer encode the documents
encoded_docs = t.texts_to_sequences(docs)
print(encoded_docs)
# pad documents to a max length of 4 words
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)
# load the whole embedding into memory
embeddings_index = dict()
f = open('../glove_data/glove.6B/glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))
# create a weight matrix for words in training docs
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=4, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
# summarize the model
print(model.summary())
# fit the model
model.fit(padded_docs, labels, epochs=50, verbose=0)
# evaluate the model
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))
```